In [313]:
%matplotlib inline
import ast
from bs4 import BeautifulSoup
import csv
import json
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial.polynomial import polyfit
import pandas as pd
import re
import requests
from scipy import stats
import seaborn as sns
import sqlite3
from sqlite3 import Error
import time
import datetime
from config import api_key_tmdb

In [314]:
def imdb_url_designer():
    movies = {"title": [], "genre": [], "imdb_score": [], "meta_score": [], "imdb_id": [], "year": []}
    url = None
    for i in range(1, 10000, 50):
        if i < 51:
            url = "https://www.imdb.com/search/title?title_type=movie&genres=action&explore=title_type,genres&ref_=adv_prv"
        if i > 50:
            number = str(i)
            url = "https://www.imdb.com/search/title?title_type=movie&genres=action&start=" + number + "&explore=title_type,genres&ref_=adv_nxt"
        r = requests.get(url)
        if r.status_code == 200:
            c = r.content
            soup = BeautifulSoup(c, 'html.parser')
            movie_articles = soup.find_all("div", class_="lister-item mode-advanced")
            for movie_article in movie_articles:
                expression = "(\\d+)"
                regex = re.compile(expression, re.IGNORECASE | re.DOTALL)
                match = regex.search(movie_article.find("span", class_="lister-item-year").text)
                if match:
                    movie_year = int(match.group(1))
                    if movie_year <= 2018:
                        movies["year"].append(movie_year) 
                        movie_name = movie_article.find("h3", class_="lister-item-header").a.text
                        movies["title"].append(movie_name)
                        movie_genre = movie_article.find("span", class_="genre").text.strip().replace("/n","").split(",")
                        movies["genre"].append(movie_genre)
                        movie_imdb_id = movie_article.find("div", class_="ribbonize")["data-tconst"]
                        movies["imdb_id"].append(movie_imdb_id)
                        movie_imdb_rating = movie_article.find("div", class_="inline-block ratings-imdb-rating")
                        if movie_imdb_rating:
                            movie_imdb_rating = movie_imdb_rating.text.strip().replace("/n","")
                            movies["imdb_score"].append(movie_imdb_rating) 
                        else:
                            movies["imdb_score"].append(" ")                             
                        movie_meta_rating = movie_article.find("span", class_="metascore ")
                        if movie_meta_rating:
                            movie_meta_rating = movie_meta_rating.text.strip()
                            movies["meta_score"].append(movie_meta_rating)
                        else:
                            movies["meta_score"].append(" ")
        else:
            print(r.status_code)
    return movies

In [47]:
# movies_dictionary = imdb_url_designer()

In [49]:
# imdb_df = pd.DataFrame.from_dict(movies_dictionary)

In [55]:
# imdb_df.to_csv(r"/Users/florencialeoni/Documents/flatironDS/Mod1/imdb_movies_data.csv")

In [315]:
# Function to dynamically make API requests to TMDb.org
def make_api_requests(movie_ids):
    api_key = api_key_tmdb
    for idx, movie in enumerate(movie_ids):
        if movie["tmdb_id"] == None:
            url = "https://api.themoviedb.org/3/find/" + movie["imdb_id"] + "?api_key=" + api_key + "&language=en-US&external_source=imdb_id"
            r = requests.get(url)
            if r.status_code == 200:
                try:
                    limit_remaining = int(r.headers['X-RateLimit-Remaining'])
                except KeyError as e:
                    print(f"KeyError: {str(e)}")
                    print(f"Response: {r}")
                    print(f"Headers: {r.headers}")

                reset_time = int(r.headers['X-RateLimit-Reset'])
                result = r.json()
                if result["movie_results"]:
                    print(f"\r{idx}", end = "")
                    movie["tmdb_id"] = result["movie_results"][0]["id"]
                else:
                    print("Result came back empty for id: " + movie["imdb_id"])
            else:
                print(r.status_code)

            while limit_remaining == 2 and reset_time > time.time():
                print("Limit reached, waiting...\r", end = "")
                time.sleep(1)
            print(f"                              ", end = "")

In [103]:
# movie_ids = list(map(lambda imdb_id : {"imdb_id": imdb_id, "tmdb_id": None }, movies_dictionary["imdb_id"]))

In [109]:
# make_api_requests(movie_ids)
# movie_ids

Result came back empty for id: tt4119054
                              Result came back empty for id: tt3781812
                              Result came back empty for id: tt5938666
                              Result came back empty for id: tt2226694
                              Result came back empty for id: tt1232828
                              Result came back empty for id: tt8363508
                              Result came back empty for id: tt5709892
                              Result came back empty for id: tt5351576
                              Result came back empty for id: tt4669408
                              Result came back empty for id: tt4730224
                              Result came back empty for id: tt4737772
                              Result came back empty for id: tt8157072
                              Result came back empty for id: tt1685674
                              Result came back empty for id: tt4669580
3852                              Re

                              Result came back empty for id: tt0331836
                              Result came back empty for id: tt8444572
                              Result came back empty for id: tt7159036
                              Result came back empty for id: tt4772986
                              Result came back empty for id: tt0288427
                              Result came back empty for id: tt0319171
                              Result came back empty for id: tt3604312
                              Result came back empty for id: tt5521836
                              Result came back empty for id: tt3628836
                              Result came back empty for id: tt7356148
                              Result came back empty for id: tt6372460
                              Result came back empty for id: tt0393896
                              Result came back empty for id: tt5132854
                              Result came back empty for id: tt0365351
      

                              Result came back empty for id: tt4625030
                              Result came back empty for id: tt5775164
                              Result came back empty for id: tt1529234
                              Result came back empty for id: tt7086410
                              Result came back empty for id: tt8829462
                              Result came back empty for id: tt5809170
                              Result came back empty for id: tt4085456
                              Result came back empty for id: tt0438216
                              Result came back empty for id: tt0187338
                              Result came back empty for id: tt6687650
                              Result came back empty for id: tt8806770
                              Result came back empty for id: tt3437996
                              Result came back empty for id: tt7000522
                              Result came back empty for id: tt3118370
      

                              Result came back empty for id: tt8757056
                              Result came back empty for id: tt5128282
                              Result came back empty for id: tt1643239
                              Result came back empty for id: tt7856408
                              Result came back empty for id: tt4641706
                              Result came back empty for id: tt4907766
                              Result came back empty for id: tt7617874
                              Result came back empty for id: tt3572642
                              Result came back empty for id: tt1350913
                              Result came back empty for id: tt6280994
                              Result came back empty for id: tt0391282
                              Result came back empty for id: tt7642904
                              Result came back empty for id: tt7733440
                              Result came back empty for id: tt2427050
      

[{'imdb_id': 'tt4633694', 'tmdb_id': 324857},
 {'imdb_id': 'tt1477834', 'tmdb_id': 297802},
 {'imdb_id': 'tt4154756', 'tmdb_id': 299536},
 {'imdb_id': 'tt4701182', 'tmdb_id': 424783},
 {'imdb_id': 'tt6491178', 'tmdb_id': 438674},
 {'imdb_id': 'tt5463162', 'tmdb_id': 383498},
 {'imdb_id': 'tt1571234', 'tmdb_id': 428078},
 {'imdb_id': 'tt1825683', 'tmdb_id': 284054},
 {'imdb_id': 'tt3501632', 'tmdb_id': 284053},
 {'imdb_id': 'tt4881806', 'tmdb_id': 351286},
 {'imdb_id': 'tt2015381', 'tmdb_id': 118340},
 {'imdb_id': 'tt4530422', 'tmdb_id': 438799},
 {'imdb_id': 'tt6998518', 'tmdb_id': 460885},
 {'imdb_id': 'tt5164214', 'tmdb_id': 402900},
 {'imdb_id': 'tt4532826', 'tmdb_id': 375588},
 {'imdb_id': 'tt1386697', 'tmdb_id': 297761},
 {'imdb_id': 'tt0848228', 'tmdb_id': 24428},
 {'imdb_id': 'tt5095030', 'tmdb_id': 363088},
 {'imdb_id': 'tt3896198', 'tmdb_id': 283995},
 {'imdb_id': 'tt1270797', 'tmdb_id': 335983},
 {'imdb_id': 'tt6133466', 'tmdb_id': 442249},
 {'imdb_id': 'tt4951982', 'tmdb_id'

In [110]:
# movie_ids_df = pd.DataFrame.from_dict(movie_ids)

In [111]:
# movie_ids_df.to_csv(r"/Users/florencialeoni/Documents/flatironDS/Mod1/movie_ids.csv")

In [321]:
# movie_ids_df = movie_ids_df.drop(movie_ids_df[movie_ids_df["tmdb_id"].isnull()].index)
# movie_ids_df


In [320]:
# movie_ids_df["tmdb_id"] = movie_ids_df["tmdb_id"].astype(int)
# tmdb_ids = movie_ids_df["tmdb_id"]
# tmdb_ids

In [319]:
# file_path = "/Users/florencialeoni/Documents/flatironDS/Mod_1_FP/mod1_movies/tmdb_ids.csv"
# tmdb_ids_series = pd.read_csv(file_path, usecols=["tmdb_id"])
# tmdb_ids_series

In [316]:
# Function to dynamically make API requests to TMDb.org
def make_movie_api_requests(movies_dict):
    api_key = "43cc392eacb0b43d61ad1782c231cd8b"  
    for idx, m_id in enumerate(movies_dict['id']):
        if movies_dict["title"][idx] == None:
            m_id = str(m_id)
            url = "https://api.themoviedb.org/3/movie/" + m_id + "?api_key=" + api_key + "&language=en-US"
            r = requests.get(url)
            if r.status_code == 200:
                try:
                    limit_remaining = int(r.headers['X-RateLimit-Remaining'])
                    reset_time = int(r.headers['X-RateLimit-Reset'])
                except KeyError as e:
                    print(f"KeyError: {str(e)}")
                    print(f"Response: {r}")
                    print(f"Headers: {r.headers}")

                result = r.json()
                print("\r", idx, end = '')
                if result["title"]:
                    movies_dict["title"][idx] = result["title"]
                else:
                    movies_dict["title"][idx] = result["title"]
                if result["budget"]:
                    movies_dict["budget"].append(result["budget"])
                else:
                    movies_dict["budget"].append(np.nan)

                if result["belongs_to_collection"]:
                    movies_dict["belongs_to_collection"].append(result["belongs_to_collection"]["name"])
                else:
                    movies_dict["belongs_to_collection"].append(np.nan) 

                if result["genres"]:
                    movies_dict["genres"].append(list(map(lambda genre: genre["name"], result["genres"])))
                else:
                    movies_dict["genres"].append(np.nan)

                if result["imdb_id"]:
                    movies_dict["imdb_id"].append(result["imdb_id"])
                else:
                    movies_dict["imdb_id"].append(np.nan)

                if result["overview"]:
                    movies_dict["overview"].append(result["overview"])
                else:
                    movies_dict["overview"].append(np.nan)

                if result["tagline"]:
                    movies_dict["tagline"].append(result["tagline"])
                else:
                    movies_dict["tagline"].append(np.nan)

                if result["runtime"]:
                    movies_dict["runtime"].append(result["runtime"])
                else:
                    movies_dict["runtime"].append(np.nan)

                if result["revenue"]:
                    movies_dict["revenue"].append(result["revenue"])
                else:
                    movies_dict["revenue"].append(np.nan)

                if result["release_date"]:
                    movies_dict["release_date"].append(result["release_date"])
                else:
                    movies_dict["release_date"].append(np.nan)

                if result["production_companies"]:
                    movies_dict["production_companies"].append(result["production_companies"][0]["name"])
                else:
                    movies_dict["production_companies"].append(np.nan)

                if result["vote_count"]:
                    movies_dict["vote_count"].append(result["vote_count"])
                else:
                    movies_dict["vote_count"].append(np.nan)

                if result["vote_average"]:
                    movies_dict["vote_average"].append(result["vote_average"])
                else:
                    movies_dict["vote_average"].append(np.nan)

                if result["popularity"]:
                    movies_dict["popularity"].append(result["popularity"])
                else:
                    movies_dict["popularity"].append(np.nan)
            else:
                print(r.status_code)
            while limit_remaining == 2 and reset_time > time.time():
                    print("Limit reached, waiting...\r", end = "")
                    time.sleep(1)

In [194]:
#tmdb_movies_dict = { "id": tmdb_ids.tolist(), "title": list(map(lambda el: None, tmdb_ids.tolist())), "belongs_to_collection": [], "budget": [], "genres": [], "imdb_id": [], 
#            "overview": [], "release_date": [], "revenue": [], "tagline": [], "release_date": [],
#            "vote_average": [], "vote_count": [], "runtime": [], "popularity": [], "production_companies": [] }


In [322]:
# make_movie_api_requests(tmdb_movies_dict)
# tmdb_movies_dict

In [318]:
# tmdb_movies_df = pd.DataFrame.from_dict(tmdb_movies_dict)
# tmdb_movies_df

,id,title,belongs_to_collection,budget,genres,imdb_id,overview,release_date,revenue,tagline,vote_average,vote_count,runtime,popularity,production_companies
0,324857,Spider-Man: Into the Spider-Verse,Spider-Man: Into the Spider-Verse Collection,90000000.0,"[Action, Adventure, Animation, Science Fiction...",tt4633694,Miles Morales is juggling his life between bei...,2018-12-07,3.628236e+08,More Than One Wears the Mask,8.5,2899.0,117.0,115.129,Columbia Pictures
1,297802,Aquaman,Aquaman Collection,160000000.0,"[Action, Adventure, Fantasy, Science Fiction]",tt1477834,Once home to the most advanced civilization on...,2018-12-07,1.143689e+09,Home Is Calling,6.8,4885.0,144.0,129.402,DC Comics
2,299536,Avengers: Infinity War,The Avengers Collection,300000000.0,"[Adventure, Action, Fantasy]",tt4154756,As the Avengers and their allies have continue...,2018-04-25,2.046240e+09,An entire universe. Once and for all.,8.3,12297.0,149.0,129.988,Marvel Studios
3,424783,Bumblebee,Transformers Collection,135000000.0,"[Action, Adventure, Science Fiction]",tt4701182,"On the run in the year 1987, Bumblebee finds r...",2018-12-15,4.203613e+08,Every Hero Has a Beginning,6.5,1586.0,114.0,226.241,Hasbro
4,438674,Dragged Across Concrete,NaN,15000000.0,"[Drama, Crime, Action, Thriller]",tt6491178,"Two policemen, one an old-timer, the other his...",2019-02-21,NaN,Those who can't earn a living must find anothe...,6.9,23.0,159.0,34.986,Cinestate
5,383498,Deadpool 2,Deadpool Collection,110000000.0,"[Action, Comedy, Adventure]",tt5463162,Wisecracking mercenary Deadpool battles the ev...,2018-05-15,7.415474e+08,Prepare for the Second Coming.,7.5,7993.0,121.0,42.911,Marvel Entertainment
6,428078,Mortal Engines,NaN,100000000.0,"[Science Fiction, Action, Adventure, Fantasy, ...",tt1571234,"Many thousands of years in the future, Earth’s...",2018-11-27,1.042365e+08,Some Scars Never Heal,5.9,1261.0,129.0,70.197,Scholastic Productions
7,284054,Black Panther,Black Panther Collection,200000000.0,"[Action, Adventure, Fantasy, Science Fiction]",tt1825683,King T'Challa returns home from America to the...,2018-02-13,1.346739e+09,Long live the king.,7.4,11333.0,134.0,60.071,Marvel Studios
8,284053,Thor: Ragnarok,Thor Collection,180000000.0,"[Action, Adventure, Comedy, Fantasy, Science F...",tt3501632,Thor is imprisoned on the other side of the un...,2017-10-25,8.539771e+08,No Hammer. No Problem.,7.5,10535.0,131.0,54.183,Marvel Studios
9,351286,Jurassic World: Fallen Kingdom,Jurassic Park Collection,170000000.0,"[Action, Adventure, Science Fiction]",tt4881806,Three years after the demise of Jurassic World...,2018-06-06,1.303460e+09,The park is gone,6.5,5545.0,129.0,37.211,Amblin Entertainment


In [225]:
# tmdb_movies_df.to_csv(r"/Users/florencialeoni/Documents/flatironDS/Mod1/tmdb_movies_df.csv")

In [286]:
# Importing the crew file
data_two = "/Users/florencialeoni/Documents/flatironDS/Mod1/Blogpost_1/the-movies-dataset/credits.csv"
use_cols_two = ["id", "cast", "crew"]

# Using pandas.read_csv method to load the file
crew_df = pd.read_csv(data_two, usecols = use_cols_two)

# Coerce the "id" column to numeric, drop the null values, and cast it as integer
crew_df["id"] = pd.to_numeric(crew_df["id"], downcast = "integer", errors = "coerce")
crew_df = crew_df.drop(crew_df[crew_df['id'].isnull()].index)
crew_df["id"] = crew_df["id"].astype(int)

# Converted both columns "crew" and "cast" to Abstract Syntax Trees,
# Using Python's ast module and the .literal_eval() function, and
# Applying it through the .apply() method. 
crew_df["crew"] = crew_df["crew"].apply(ast.literal_eval)
crew_df["cast"] = crew_df["cast"].apply(ast.literal_eval)

# Helper function to extract Directors' names from the recently parsed "crew" column
def extract_director(crew_list):
    for member in crew_list:
        if member["department"] == "Directing" and member["job"] == "Director":
            return member["name"]
crew_df["director_name"] = crew_df["crew"].map(extract_director)

# Helper function to extract Directors' genders from the recently parsed "crew" column
def extract_director_gender(crew_list):
    for member in crew_list:
        if member["department"] == "Directing" and member["job"] == "Director":
            return member["gender"]
crew_df["director_gender"] = crew_df["crew"].map(extract_director_gender)

#Helper function to extract Leading Actor from the recently parsed "cast" column
def extract_leading_actor(cast_list):
    for member in cast_list:
        if member["order"] == 0:
            return member["name"]
crew_df["leading_actor_name"] = crew_df["cast"].map(extract_leading_actor)

# Helper function to extract Supporting Actor from the recently parsed "cast" column
def extract_supporting_actor(cast_list):
    for member in cast_list:
        if member["order"] == 1:
            return member["name"]
crew_df["supporting_actor_name"] = crew_df["cast"].map(extract_supporting_actor)

# Drop the original "cast" and "crew" columns since we already have
# The information we needed for the analysis.
crew_df = crew_df.drop(["cast"], axis = 1)
crew_df = crew_df.drop(["crew"], axis = 1)
crew_df

,id,director_name,director_gender,leading_actor_name,supporting_actor_name
0,862,John Lasseter,2.0,Tom Hanks,Tim Allen
1,8844,Joe Johnston,2.0,Robin Williams,Jonathan Hyde
2,15602,Howard Deutch,2.0,Walter Matthau,Jack Lemmon
3,31357,Forest Whitaker,2.0,Whitney Houston,Angela Bassett
4,11862,Charles Shyer,2.0,Steve Martin,Diane Keaton
5,949,Michael Mann,2.0,Al Pacino,Robert De Niro
6,11860,Sydney Pollack,2.0,Harrison Ford,Julia Ormond
7,45325,Peter Hewitt,2.0,Jonathan Taylor Thomas,Brad Renfro
8,9091,Peter Hyams,2.0,Jean-Claude Van Damme,Powers Boothe
9,710,Martin Campbell,2.0,Pierce Brosnan,Sean Bean


In [288]:
# Merge the main DataFrame with the crew DataFrame 
tmdb_movies_cast_df = pd.merge(tmdb_movies_df, crew_df, how='left', on=["id"])

# Set the "id" column as the Index for the DataFrame
tmdb_movies_cast_df = tmdb_movies_cast_df.set_index("id")

In [317]:
tmdb_movies_cast_df

,title,belongs_to_collection,budget,genres,imdb_id,overview,release_date,revenue,tagline,vote_average,vote_count,runtime,popularity,production_companies,director_name,director_gender,leading_actor_name,supporting_actor_name
id,,,,,,,,,,,,,,,,,,
324857,Spider-Man: Into the Spider-Verse,Spider-Man: Into the Spider-Verse Collection,90000000.0,"[Action, Adventure, Animation, Science Fiction...",tt4633694,Miles Morales is juggling his life between bei...,2018-12-07,3.628236e+08,More Than One Wears the Mask,8.5,2899.0,117.0,115.129,Columbia Pictures,NaN,NaN,NaN,NaN
297802,Aquaman,Aquaman Collection,160000000.0,"[Action, Adventure, Fantasy, Science Fiction]",tt1477834,Once home to the most advanced civilization on...,2018-12-07,1.143689e+09,Home Is Calling,6.8,4885.0,144.0,129.402,DC Comics,NaN,NaN,NaN,NaN
299536,Avengers: Infinity War,The Avengers Collection,300000000.0,"[Adventure, Action, Fantasy]",tt4154756,As the Avengers and their allies have continue...,2018-04-25,2.046240e+09,An entire universe. Once and for all.,8.3,12297.0,149.0,129.988,Marvel Studios,NaN,NaN,NaN,NaN
424783,Bumblebee,Transformers Collection,135000000.0,"[Action, Adventure, Science Fiction]",tt4701182,"On the run in the year 1987, Bumblebee finds r...",2018-12-15,4.203613e+08,Every Hero Has a Beginning,6.5,1586.0,114.0,226.241,Hasbro,NaN,NaN,NaN,NaN
438674,Dragged Across Concrete,NaN,15000000.0,"[Drama, Crime, Action, Thriller]",tt6491178,"Two policemen, one an old-timer, the other his...",2019-02-21,NaN,Those who can't earn a living must find anothe...,6.9,23.0,159.0,34.986,Cinestate,NaN,NaN,NaN,NaN
383498,Deadpool 2,Deadpool Collection,110000000.0,"[Action, Comedy, Adventure]",tt5463162,Wisecracking mercenary Deadpool battles the ev...,2018-05-15,7.415474e+08,Prepare for the Second Coming.,7.5,7993.0,121.0,42.911,Marvel Entertainment,NaN,NaN,NaN,NaN
428078,Mortal Engines,NaN,100000000.0,"[Science Fiction, Action, Adventure, Fantasy, ...",tt1571234,"Many thousands of years in the future, Earth’s...",2018-11-27,1.042365e+08,Some Scars Never Heal,5.9,1261.0,129.0,70.197,Scholastic Productions,NaN,NaN,NaN,NaN
284054,Black Panther,Black Panther Collection,200000000.0,"[Action, Adventure, Fantasy, Science Fiction]",tt1825683,King T'Challa returns home from America to the...,2018-02-13,1.346739e+09,Long live the king.,7.4,11333.0,134.0,60.071,Marvel Studios,NaN,NaN,NaN,NaN
284053,Thor: Ragnarok,Thor Collection,180000000.0,"[Action, Adventure, Comedy, Fantasy, Science F...",tt3501632,Thor is imprisoned on the other side of the un...,2017-10-25,8.539771e+08,No Hammer. No Problem.,7.5,10535.0,131.0,54.183,Marvel Studios,Taika Waititi,2.0,Chris Hemsworth,Tom Hiddleston


In [312]:
# tmdb_movies_cast_df.to_csv(r"/Users/florencialeoni/Documents/flatironDS/Mod_1_FP/mod1_movies/tmdb_movies_cast_df.csv")